In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType, IntegerType,TimestampType
# Build SparkSession
import getpass
spark = SparkSession.builder.master("local[*]").appName("ETL Pipeline")\
        .config("spark.jars", "../jars/postgresql-42.5.1.jar").getOrCreate()

# Set Logging Level to WARN
spark.sparkContext.setLogLevel("WARN")

23/01/24 16:35:13 WARN Utils: Your hostname, qtt-HP-EliteBook-840-G6 resolves to a loopback address: 127.0.1.1; using 192.168.1.35 instead (on interface wlp58s0)
23/01/24 16:35:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/24 16:35:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
source_data_directory = "./financial_data/"
economic_indicators=["economic_indicatorCPI",
                     "economic_indicatorCONSUMER_SENTIMENT",
                     "economic_indicatorDURABLES",
                     "economic_indicatorINFLATION_EXPECTATION",
                     "economic_indicatorNONFARM_PAYROLL",
                     "economic_indicatorRETAIL_SALES",
                     "economic_indicatorTREASURY_YIELD",
                     "economic_indicatorUNEMPLOYMENT"]
# Define csv input schema
schema = StructType([
    StructField("timestamp", DateType()),
    StructField("value", DecimalType(precision=38, scale=2)),
])


In [3]:
dictionary_df={}
for i in economic_indicators:
    dictionary_df[i] = spark.read.option("header", "true").csv(source_data_directory+i+".csv", schema=schema)

In [4]:
for i in economic_indicators:
    dictionary_df[i].createOrReplaceTempView(i)
    

In [5]:
dataDF=spark.sql("select economic_indicatorCPI.timestamp as date, economic_indicatorCPI.value as CPI_value, economic_indicatorCONSUMER_SENTIMENT.value as CONSUMER_SENTIMENT_value,"+
                 "economic_indicatorDURABLES.value as DURABLES_value,economic_indicatorINFLATION_EXPECTATION.value as INFLATION_EXPECTATION_value,"+
                 "economic_indicatorNONFARM_PAYROLL.value as NONFARM_PAYROLL_value,economic_indicatorRETAIL_SALES.value as RETAIL_SALES_value,"+
                 "economic_indicatorTREASURY_YIELD.value as TREASURY_YIELD_value,economic_indicatorUNEMPLOYMENT.value as UNEMPLOYMENT_value "
          "from economic_indicatorCPI LEFT OUTER JOIN economic_indicatorCONSUMER_SENTIMENT ON economic_indicatorCPI.timestamp == economic_indicatorCONSUMER_SENTIMENT.timestamp "
          "LEFT OUTER JOIN economic_indicatorDURABLES ON economic_indicatorCPI.timestamp == economic_indicatorDURABLES.timestamp "+
          "LEFT OUTER JOIN economic_indicatorINFLATION_EXPECTATION ON economic_indicatorCPI.timestamp == economic_indicatorINFLATION_EXPECTATION.timestamp "+
          "LEFT OUTER JOIN economic_indicatorTREASURY_YIELD ON economic_indicatorCPI.timestamp == economic_indicatorTREASURY_YIELD.timestamp "+
          "LEFT OUTER JOIN economic_indicatorNONFARM_PAYROLL ON economic_indicatorCPI.timestamp == economic_indicatorNONFARM_PAYROLL.timestamp "+
          "LEFT OUTER JOIN economic_indicatorRETAIL_SALES ON economic_indicatorCPI.timestamp == economic_indicatorRETAIL_SALES.timestamp "+
          "LEFT OUTER JOIN economic_indicatorUNEMPLOYMENT ON economic_indicatorCPI.timestamp == economic_indicatorUNEMPLOYMENT.timestamp ")

In [6]:
dataDF.count()

1318

In [7]:
dataDF.show()

+----------+---------+------------------------+--------------+---------------------------+---------------------+------------------+--------------------+------------------+
|      date|CPI_value|CONSUMER_SENTIMENT_value|DURABLES_value|INFLATION_EXPECTATION_value|NONFARM_PAYROLL_value|RETAIL_SALES_value|TREASURY_YIELD_value|UNEMPLOYMENT_value|
+----------+---------+------------------------+--------------+---------------------------+---------------------+------------------+--------------------+------------------+
|2022-10-01|   298.01|                    null|          null|                       null|            154369.00|         597492.00|                3.98|              3.70|
|2022-09-01|   296.81|                   58.60|     290199.00|                       4.70|            153197.00|         576853.00|                3.52|              3.50|
|2022-08-01|   296.17|                   58.20|     279323.00|                       4.80|            152674.00|         613416.00|         

In [8]:
from pyspark.sql.functions import udf
def extract_month(date):
    if date is not None:
        return int(date.month)


def extract_year(date):
    if date is not None:
        return int(date.year)


def extract_day(date):
    if date is not None:
        return int(date.day)
udf_month = udf(extract_month, IntegerType())
udf_year = udf(extract_year, IntegerType())
udf_day = udf(extract_day, IntegerType())

In [9]:
day_month_year_DF = dataDF \
    .withColumn("month", udf_month("date")) \
    .withColumn("year", udf_year("date")) \
    .withColumn("day", udf_day("date"))
day_month_year_DF.show()

+----------+---------+------------------------+--------------+---------------------------+---------------------+------------------+--------------------+------------------+-----+----+---+
|      date|CPI_value|CONSUMER_SENTIMENT_value|DURABLES_value|INFLATION_EXPECTATION_value|NONFARM_PAYROLL_value|RETAIL_SALES_value|TREASURY_YIELD_value|UNEMPLOYMENT_value|month|year|day|
+----------+---------+------------------------+--------------+---------------------------+---------------------+------------------+--------------------+------------------+-----+----+---+
|2022-10-01|   298.01|                    null|          null|                       null|            154369.00|         597492.00|                3.98|              3.70|   10|2022|  1|
|2022-09-01|   296.81|                   58.60|     290199.00|                       4.70|            153197.00|         576853.00|                3.52|              3.50|    9|2022|  1|
|2022-08-01|   296.17|                   58.20|     279323.00|   

In [10]:
finalDF=day_month_year_DF.drop("date")


In [11]:
finalDF.printSchema()


root
 |-- CPI_value: decimal(38,2) (nullable = true)
 |-- CONSUMER_SENTIMENT_value: decimal(38,2) (nullable = true)
 |-- DURABLES_value: decimal(38,2) (nullable = true)
 |-- INFLATION_EXPECTATION_value: decimal(38,2) (nullable = true)
 |-- NONFARM_PAYROLL_value: decimal(38,2) (nullable = true)
 |-- RETAIL_SALES_value: decimal(38,2) (nullable = true)
 |-- TREASURY_YIELD_value: decimal(38,2) (nullable = true)
 |-- UNEMPLOYMENT_value: decimal(38,2) (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- day: integer (nullable = true)



In [26]:
driver = "org.postgresql.Driver"
url='jdbc:postgresql://bigdata-database.ce0q5dfktsor.us-east-1.rds.amazonaws.com/postgres'  
dbtable='public.financial'
user='postgres'
password='truong157359'

In [27]:
finalDF.write.format('jdbc').options(
      url=url,
      driver=driver,
      dbtable=dbtable,
      user=user,
      password=password).mode('append').save() 